<a href="https://colab.research.google.com/github/jetsonmom/roboflow-test/blob/main/roboflow0607.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# Git clone method (for development)

%cd {HOME}
!git clone github.com/ultralytics/ultralytics
%cd {HOME}/ultralytics
!pip install -e .

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

cli 예제_dog detection picture

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
%cd {HOME}
Image(filename='runs/detect/predict/dog.jpeg', height=600)

🐍 Python SDK

In [ ]:
model = YOLO(f'{HOME}/yolov8n.pt')
results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
results[0].boxes.conf

In [ ]:
results[0].boxes.cls

축구선수 디텍트

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
dataset = project.version(1).download("yolov8")

custom detect training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

results

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict3/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

Roboflow에 모델 배포
YOLOv8 모델 학습을 마치면 사용할 수 있는 학습된 가중치 세트가 준비됩니다. 이러한 가중치는 프로젝트의 /runs/Detect/train/weights/best.pt 폴더에 있습니다. 모델 가중치를 Roboflow Deploy에 업로드하여 무한히 확장 가능한 인프라에서 훈련된 가중치를 사용할 수 있습니다.

Roboflow pip 패키지의 .deploy() 함수는 이제 YOLOv8 가중치 업로드를 지원합니다.

모델 가중치를 업로드하려면 앞서 언급한 노트북의 "사용자 정의 모델 추론" 섹션에 다음 코드를 추가하세요.

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train/")

In [ ]:
#While your deployment is processing, checkout the deployment docs to take your model to most destinations https://docs.roboflow.com/inference

In [ ]:
#Run inference on your model on a persistant, auto-scaling, cloud API

#load model
model = project.version(dataset.version).model

#choose random test set image
import os, random
test_set_loc = dataset.location + "/test/images/"
random_test_image = random.choice(os.listdir(test_set_loc))
print("running inference on " + random_test_image)

pred = model.predict(test_set_loc + random_test_image, confidence=40, overlap=30).json()
pred

In [ ]:
import os
import shutil
import glob
import moviepy.editor as mp


# HOME 경로 설정
HOME = "/content"


# YOLO 명령어 실행
os.chdir(HOME)
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt conf=0.05 source=/content/data/test.mp4 save=True


# 출력 디렉토리 찾기
output_dirs = sorted(glob.glob(os.path.join(HOME, "runs", "detect", "predict*")), key=os.path.getmtime, reverse=True)
latest_output_dir = output_dirs[0] if output_dirs else None


if latest_output_dir:
    # 결과 파일 경로
    output_file = os.path.join(latest_output_dir, "test.mp4")
    target_file = "/content/data/test1-1/test/test_output.mp4"

    if os.path.exists(output_file):
        # .avi 파일을 .mp4 파일로 변환
        video_clip = mp.VideoFileClip(output_file)
        video_clip.write_videofile(target_file, codec='libx264')
        print(f"Processed video saved as {target_file}")
    else:
        print("Error: The output video file was not found.")
else:
    print("Error: The output directory was not found.")
